In [1]:
import qcodes as qc
station = qc.Station(config_file='config/station.yaml')

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /Users/caijiaqi/.qcodes/logs/command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : /Users/caijiaqi/.qcodes/logs/250929-40687-qcodes.log


In [2]:
dac1 = station.load_mock_dac1()
dac2 = station.load_mock_dac2()

In [3]:
%load_ext instrmcp.extensions

✅ QCoDeS MCP extension loaded
🛡️  Default mode: safe
📋 Use %mcp_status to check server status
⚠️  Use %mcp_unsafe to switch to unsafe mode (if needed)
🚀 Use %mcp_start to start the server


In [4]:
%mcp_status

🛡️ MCP Server Status:
   Desired Mode: safe
   Server Instance: ❌ Not created yet
   Available: %mcp_start


In [5]:
%mcp_option add measureit database

✅ Added: measureit
✅ Added: database
✅ Changes will take effect when server starts


In [6]:
%mcp_unsafe

⚠️  Mode set to unsafe
⚠️  UNSAFE MODE: execute_editing_cell tool will be available
✅ Mode will take effect when server starts


In [7]:
%mcp_start

🚀 Starting MCP server...
🚀 QCoDeS MCP Server running on http://127.0.0.1:8123
🔑 Access token: se-YcafhZN9HV-wVc89hXRLJfYZD5fTigxcJY7ciI5w


2025-09-29 18:18:52,751 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 110 ¦ /Users/caijiaqi/miniforge3/envs/instrMCPdev/lib/python3.11/site-packages/websockets/legacy/__init__.py:6: DeprecationWarning: websockets.legacy is deprecated; see https://websockets.readthedocs.io/en/stable/howto/upgrade.html for upgrade instructions
  warnings.warn(  # deprecated in 14.0 - 2024-11-09

2025-09-29 18:18:52,756 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 110 ¦ /Users/caijiaqi/miniforge3/envs/instrMCPdev/lib/python3.11/site-packages/uvicorn/protocols/websockets/websockets_impl.py:17: DeprecationWarning: websockets.server.WebSocketServerProtocol is deprecated
  from websockets.server import WebSocketServerProtocol

INFO:     Started server process [40687]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8123 (Press CTRL+C to quit)


✅ MCP server started in ⚠️ unsafe mode
⚠️  UNSAFE MODE: execute_editing_cell tool is available
INFO:     127.0.0.1:51389 - "DELETE /mcp HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:51391 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51391 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:51391 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51389 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51389 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51389 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51445 - "POST /mcp HTTP/1.1" 200 OK


2025-09-29 18:19:15,094 ¦ instrmcp.servers.jupyter_qcodes.registrars.resources ¦ ERROR ¦ resources ¦ station_state ¦ 100 ¦ Error generating station_state resource: 'QCodesReadOnlyTools' object has no attribute 'get_station_snapshot'


INFO:     127.0.0.1:51445 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51445 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51445 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51445 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51445 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51445 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51527 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51559 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51559 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51649 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51649 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51734 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51807 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51861 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51918 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51953 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51987 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51987 - "POST /mcp HTTP/1.1"

In [8]:
sadfasdfasdfasdf

NameError: name 'sadfasdfasdfasdf' is not defined

In [9]:
1+1

2